In [1]:
from transformers import pipeline
import nltk
import re

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
unmasker = pipeline('fill-mask', model = 'FacebookAI/xlm-roberta-large')

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of the model checkpoint at FacebookAI/xlm-roberta-large were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
from nltk import word_tokenize

file = open("/kaggle/input/spell-data/tamil_corpus.txt", "r", encoding="utf-8")
corpus = file.read()
corpus = word_tokenize(corpus)
len(corpus)

1460432

In [ ]:
import pandas as pd
import numpy as np

textFile = pd.read_excel("/kaggle/input/spell-data/test_tamil_indic.xlsx")
print(textFile.shape)
textFile.head()

(1580, 8)


,predictions,transcripts,WER,CER,Unnamed: 4,Unnamed: 5,Average WER,Average CER
0,சப்தம்பர் இருபத்தி நான்காம் தேதி வியாழன் பக்ரீத்,செப்டம்பர் இருபத்தி நான்காம் தேதி வியாழன் பக்ரீத்,0.166667,0.040816,NaN,NaN,46.360215,16.724491
1,ஒரு சிலர் தனக்கு முதல் முதலில் கல்வி கற்பித்த ...,ஒரு சிலர் தனக்கு முதன் முதலில் கல்வி கற்பித்த ...,0.083333,0.010526,NaN,NaN,NaN,NaN
2,y in Australia in a Nolayamuyanra Illangaiyarg...,ஆஸ்த்திரேலியாவினுள் நுழைய முயன்ற இலங்கையர்கள் ...,1.833333,1.015152,NaN,NaN,NaN,NaN
3,yசமுதாயத்தின் நலனுக்காக பாடுபடுவார்,சமுதாயத்தின் நலனுக்காக பாடுபடுவார்,0.333333,0.029412,NaN,NaN,NaN,NaN
4,எனக்கு ஜோடியாக இரண்டு கதாநாய்கள் நடிக்கிறார்கள்,எனக்கு ஜோடியாக இரண்டு கதாநாயகிகள் நடிக்கிறார்கள்,0.200000,0.041667,NaN,NaN,NaN,NaN


In [ ]:
#removing punctuations from the text
punc = ['।', "  ","|" ,'\'', "‘",'!', '"', ',','$', '#', '@', '%', '^', '&', '*', '(', ')', '_', '-', '+', '=', '{','}', '.', ':', '?', '/', '\\', ';']

def remove_punc(txt):
    new_txt = ''
    for letter in txt:
        if (letter in punc) or (ord(letter)>=65 and ord(letter)<=90) or (ord(letter)>=97 and ord(letter)<=122):
            continue
        else:
            new_txt += letter
    return new_txt

#check whether the word is present in the dictionary or not #find the misspelt words
puncAndNum = ['1','2','3','4','5','6','7','8','9','0', '०', '१', '२', '	३', '४', '५', '६','७','८','९','।', "|" ,'\'', '!', '"', ',','$', '#', '@', '%', '^', '&', '*', '(', ')', '_', '-', '+', '=', '{','}', '.', ':', '?', '/', '\\', ';', '[', ']']

def check_misspelt(word):
    word = remove_punc(word)
    if(word not in corpus):
      if((word not in punc) and (word not in puncAndNum) and (re.match(r'[^a-zA-Z]', word[0])) and (re.match(r'[^0-9]', word[0])) and (re.match(r'[^०१२३४५६७८९]', word[0]))):
        return 1
    return 0

## model

In [ ]:
pip install -q fasttext

Note: you may need to restart the kernel to use updated packages.


In [ ]:
l1 = ['படங்கள்','பெண்கள்','பேர்','படத்தில்']
l2 = ['கதாநாயகர்கள்', 'கதாநாயகிகள்', 'கோநாய்கள்', 'கதாநாயகர்', 'கதாநாயகன்']

for x in l2:
    print("")
    for y in l1:
        vector1 = model.get_word_vector(y)
        vector2 = model.get_word_vector(x)
        cosine_similarity = np.dot(vector1, vector2) / (norm(vector1) * norm(vector2))
        print(cosine_similarity)


0.4312075
0.26574695
0.22633012
0.46444136

0.39589182
0.37598714
0.25732738
0.5478229

0.29700816
0.3181407
0.2300392
0.14994265

0.32462096
0.14329173
0.14243236
0.44831428

0.30537564
0.15851647
0.1167107
0.49765548


In [ ]:
import difflib
import fasttext
import numpy as np
from numpy.linalg import norm

model = fasttext.load_model('/kaggle/input/fasttext/indicnlp.ft.ta.300.bin')

rm_list = []
new_preds = []
count = 1

for sent in textFile['predictions']:
    print(count)
    count+=1

    sent = remove_punc(sent)
    sent_tokens = nltk.word_tokenize(sent)

    for word in sent_tokens:
        if(check_misspelt(word)):
            sent_masked = sent.replace(word, "<mask>")
            masked = unmasker(sent_masked)

            closest_match = difflib.get_close_matches(word, corpus, n=8, cutoff=0.7)

            if type(masked[0])==list:
                rm_list.append(count-2)
                continue

            if len(closest_match)==0:
                continue

            flag=0

            for x in masked:
                if x['token_str'] in closest_match:
                    sent = sent.replace(word,x['token_str'])
                    print(word,x['token_str'])
                    flag=1
                    break

            if flag==0:
                final_list = []

                for x in closest_match:
                    sim = 0

                    for y in masked:
                        vector1 = model.get_word_vector(y['token_str'])
                        vector2 = model.get_word_vector(x)

                        cosine_similarity = np.dot(vector1, vector2) / (norm(vector1) * norm(vector2))
                        sim+= cosine_similarity

                    final_list.append(sim/len(masked))

                final_list = np.array(final_list)
                z = np.argmax(final_list)
                sent = sent.replace(word,closest_match[z])
                print(word,closest_match[z])

    new_preds.append(sent)

1
சப்தம்பர் செப்தம்பர்
2
3
4
5
கதாநாய்கள் கதாநாயகிகள்
6
இசலா இசுலாய
இசமேப் செமேபர்
7
நடலமன்றோருப்பினர் சட்டமன்றஉறுப்பினர்
8
சீதைக்கிக்கோயில் மையக்கிழக்கில்
9
வெளியகம் வெளியாகும்
10
11
நோசாவின் நிசாவின்
செற்கைக்கூல் செயற்கைகோள்களில்
விழ்ந்தது வீழ்ந்தது
12
13
வழைகுடாவில் வளைகுடாவினால்
முரியடித்தது முறியடித்ததும்
மலேஷயக் மலேசியக்
14
தம்பதியார் தம்பதியராவர்
15
16
முநோகரன் முகர்
17
மதினட்பமானது திட்பமான
அருதப்படுவது கருதப்படுவது
18
பாரகிகளுக்கு பாரோக்களுக்கு
மீத்தியனாகும் அத்தியாகும்
19
20
21
துனைச்சியாவின் தென்னாசியாவின்
எதிர்கைச்சுத்தலைவர் எதிர்க்கட்சித்தலைவர்
22
கர்னாஸ் கர்னாட்
23
ம்போது அம்பது
24
மாலத்திவில் மாத்தில்
25
உதவியக்குனர் வியக்குடன்
ஜெய்க்குடிப்படத்தையுக்குகிறார் கட்டிப்பிடித்துக்கொள்கிறான்
26
அம்பதான் அம்பாதுவான்
27
சிரிதேவி சிரிமதி
சிருத்திகாசனுடன் விருத்தியுடன்
28
அணிகன் அணின்
இளக்கத்தை இரக்கத்தை
தவண்டவில்லை வேண்டுவதில்லை
குறிப்பிடுத்தக்கள் குறிப்பிடத்தகவர்கள்
29
வாடிக்குயாளர்களை வாடிக்கையாளார்களை
வீத்துக் வைத்துக்
30
வெளியேற்றை வெளியேற்றியதை
பின்னரை பொன்னரை
வாய்வை வாழ்வை


In [ ]:
textFile.drop(rm_list)
textFile['xlm_roberta_predictions'] = new_preds
textFile.to_excel("/kaggle/working/test_tamil_indic_xlm_roberta_preds.xlsx")

## difflib

In [ ]:
import difflib

new_preds = []
count = 1

for sent in textFile['predictions']:
    print(count)
    count+=1
    sent = remove_punc(sent)
    sent_tokens = nltk.word_tokenize(sent)
    for word in sent_tokens:
        if(check_misspelt(word)):
            closest_match = difflib.get_close_matches(word, corpus, n=1, cutoff=0.8)
            if len(closest_match)!=0:
                sent = sent.replace(word,closest_match[0])
                print(word,closest_match[0])
    new_preds.append(sent)

In [ ]:
textFile['difflib_predictions_1'] = new_preds
textFile.to_excel("C:/D/VIT CHN/Internships/Tamil ASR/Data/test_tamil_indic_difflib_preds.xlsx")

## Evaluate

In [ ]:
pip install -q evaluate jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import evaluate
import pandas as pd

metric1 = evaluate.load("wer")
metric2 = evaluate.load("cer")

test_df = pd.read_excel("/kaggle/working/test_tamil_indic_xlm_roberta_preds.xlsx")

final_wer = 0
final_cer = 0

WER_list = []
CER_list = []

rm = []

for i in range(test_df.shape[0]):
    tr = test_df['transcripts'][i]
    pr = test_df['xlm_roberta_predictions'][i]

    if type(pr)==float or type(tr)==float:
        rm.append(i)
        continue
    elif len(pr)==0 or len(tr)==0:
        rm.append(i)
        continue

    wer = metric1.compute(references=[tr], predictions=[pr])
    final_wer += wer
    WER_list.append(wer)

    cer = metric2.compute(references=[tr], predictions=[pr])
    final_cer += cer
    CER_list.append(cer)

    print(f'{i+1} : {100*wer}')
    print(f'{i+1} : {100*cer}')
    print("")

print("WER : ",100*(final_wer/test_df.shape[0]))
print("CER : ",100*(final_cer/test_df.shape[0]))

test_df = test_df.drop(rm)

test_df['new_WER'] = WER_list
test_df['new_CER'] = CER_list

test_df.to_excel("/kaggle/working/test_tamil_indic_xlm_roberta_preds_metrics.xlsx")